In [1]:
!pip install transformers==4.26.0

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score

# imported packages correctly?
print("successfully!")

# Load the data from the CSV file
data = pd.read_csv('/datasets/summaries/data.csv')
print("Read file")

# sample only 20% of the data
data = data.sample(frac=0.1)
print("Got sample")

# drop missing values
data.dropna(inplace=True)
print("Dropped empty values")

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)
print("split the data sets")

# Extract the summaries and full stories from the training data
train_summaries = train_data['Summary']
train_stories = train_data['Content']
print("extracted specific columns from training data")

# Extract the summaries and full stories from the testing data
test_summaries = test_data['Summary']
test_stories = test_data['Content']
print("extracted specific columns from testing data")

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("created the tokenizer")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
print("defined the model")

# Convert the data into input format for the model
train_inputs = tokenizer.batch_encode_plus(train_stories.tolist(), max_length=512, pad_to_max_length=True, return_tensors='pt')
test_inputs = tokenizer.batch_encode_plus(test_stories.tolist(), max_length=512, pad_to_max_length=True, return_tensors='pt')
print("tokenizied the training and the testing data")

# Train the model on the training data
model.train()
print("trained the model")
model.zero_grad()
print("set all the gradients of the model's parameters to zero")
outputs = model(input_ids=train_inputs['input_ids'], labels=train_summaries)
print("defined the outputs")
loss, logits = outputs[:2]
loss.backward()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
optimizer.step()

# Test the model on the testing data
model.eval()
outputs = model(input_ids=test_inputs['input_ids'])
predictions = outputs[0]

# Calculate the accuracy of the model
accuracy = accuracy_score(test_summaries, predictions)
print('Accuracy:', accuracy)


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-26 13:05:41.369756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 13:05:41.488285: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-26 13:05:41.493064: W tensorflow/stream_executor/platform/default/dso_loader.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=36b8a784-1bd6-47ec-836d-38cd0b38ef5c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>